<div style="text-align: right"><b> Ana Valentina López Chacón </b></div>
<div style="text-align: right"><b> Traducción Automática, 2024 - 2025 </b></div>

# **Python a Ingles: NLLB Finetuning**

In [1]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q datasets jiwer tqdm pandas
!pip install -q torch torchvision torchaudio
!pip install -q evaluate
!pip install -q transformers==4.47.0
!pip install -q accelerate==0.27.0
!pip install -q peft
!pip install -q bitsandbytes==0.45.0
!pip install -q sacrebleu
!pip install -q unbabel-comet
!pip install -q rouge_score

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

dataset = load_dataset("code_x_glue_ct_code_to_text", "python")

dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 251820
    })
    validation: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 13914
    })
    test: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 14918
    })
})

In [3]:
dataset['train'].features

{'id': Value(dtype='int32', id=None),
 'repo': Value(dtype='string', id=None),
 'path': Value(dtype='string', id=None),
 'func_name': Value(dtype='string', id=None),
 'original_string': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'code': Value(dtype='string', id=None),
 'code_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'docstring': Value(dtype='string', id=None),
 'docstring_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'sha': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None)}

In [4]:
print(len(dataset['train']))
print(len(dataset['validation']))
print(len(dataset['test']))

251820
13914
14918


In [5]:
seed = 42

train_dataset = dataset['train'].shuffle(seed=seed).select(range(200))
validation_dataset = dataset['validation'].shuffle(seed=seed).select(range(1000))
test_dataset = dataset['test'].shuffle(seed=seed).select(range(1000))

print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(validation_dataset))
print("Test dataset size:", len(test_dataset))

Training dataset size: 200
Validation dataset size: 1000
Test dataset size: 1000


In [6]:
train_dataset[0]['code']

'def metapolicy(request, permitted, domains=None):\n    """\n    Serves a cross-domain policy which can allow other policies\n    to exist on the same domain.\n\n    Note that this view, if used, must be the master policy for the\n    domain, and so must be served from the URL ``/crossdomain.xml`` on\n    the domain: setting metapolicy information in other policy files\n    is forbidden by the cross-domain policy specification.\n\n    **Required arguments:**\n\n    ``permitted``\n        A string indicating the extent to which other policies are\n        permitted. A set of constants is available in\n        ``flashpolicies.policies``, defining acceptable values for\n        this argument.\n\n    **Optional arguments:**\n\n    ``domains``\n        A list of domains from which to allow access. Each value may\n        be either a domain name (e.g., ``example.com``) or a wildcard\n        (e.g., ``*.example.com``). Due to serious potential security\n        issues, it is strongly recommen

In [7]:
train_dataset[0]["docstring"]

'Serves a cross-domain policy which can allow other policies\n    to exist on the same domain.\n\n    Note that this view, if used, must be the master policy for the\n    domain, and so must be served from the URL ``/crossdomain.xml`` on\n    the domain: setting metapolicy information in other policy files\n    is forbidden by the cross-domain policy specification.\n\n    **Required arguments:**\n\n    ``permitted``\n        A string indicating the extent to which other policies are\n        permitted. A set of constants is available in\n        ``flashpolicies.policies``, defining acceptable values for\n        this argument.\n\n    **Optional arguments:**\n\n    ``domains``\n        A list of domains from which to allow access. Each value may\n        be either a domain name (e.g., ``example.com``) or a wildcard\n        (e.g., ``*.example.com``). Due to serious potential security\n        issues, it is strongly recommended that you not use wildcard\n        domain values.'

In [8]:
max_tok_length = 1024

from transformers import AutoTokenizer

checkpoint = "facebook/nllb-200-distilled-600M"
src_code = "eng_Latn"
tgt_code = "eng_Latn"
tokenizer = AutoTokenizer.from_pretrained(
    checkpoint,
    padding=True,
    pad_to_multiple_of=8,
    src_lang=src_code,
    tgt_lang=tgt_code,
    truncation=True,
    max_length=max_tok_length,
    )

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [9]:
def preprocess_function(sample):
    model_inputs = tokenizer(
        sample["code"],
        text_target = sample["docstring"],
        )
    return model_inputs

In [10]:
train_dataset_filtered = train_dataset.filter(lambda x: len(preprocess_function(x)["input_ids"]) <= max_tok_length and len(preprocess_function(x)["labels"]) <= max_tok_length,
                                               desc=f"Discarding source and target sentences with more than {max_tok_length} tokens")
validation_dataset_filtered = validation_dataset.filter(lambda x: len(preprocess_function(x)["input_ids"]) <= max_tok_length and len(preprocess_function(x)["labels"]) <= max_tok_length,
                                               desc=f"Discarding source and target sentences with more than {max_tok_length} tokens")
test_dataset_filtered = test_dataset.filter(lambda x: len(preprocess_function(x)["input_ids"]) <= max_tok_length and len(preprocess_function(x)["labels"]) <= max_tok_length,
                                               desc=f"Discarding source and target sentences with more than {max_tok_length} tokens")

Discarding source and target sentences with more than 1024 tokens:   0%|          | 0/200 [00:00<?, ? examples…

Token indices sequence length is longer than the specified maximum sequence length for this model (1213 > 1024). Running this sequence through the model will result in indexing errors


Discarding source and target sentences with more than 1024 tokens:   0%|          | 0/1000 [00:00<?, ? example…

Discarding source and target sentences with more than 1024 tokens:   0%|          | 0/1000 [00:00<?, ? example…

In [11]:
print("Training dataset size:", len(train_dataset_filtered))
print("Validation dataset size:", len(validation_dataset_filtered))
print("Test dataset size:", len(test_dataset_filtered))

Training dataset size: 199
Validation dataset size: 981
Test dataset size: 978


In [12]:
tokenized_train_dataset = train_dataset_filtered.map(preprocess_function, batched=True)
tokenized_validation_dataset = validation_dataset_filtered.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset_filtered.map(preprocess_function, batched=True)

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Map:   0%|          | 0/981 [00:00<?, ? examples/s]

Map:   0%|          | 0/978 [00:00<?, ? examples/s]

In [13]:
dic = {}
for sample in tokenized_train_dataset:
    sample_length = len(sample['input_ids'])
    if sample_length not in dic:
        dic[sample_length] = 1
    else:
        dic[sample_length] += 1

for i in range(1,max_tok_length+1):
    if i in dic:
        print(f"{i:>2} {dic[i]:>3}")

50   1
51   1
54   1
55   2
57   1
58   1
59   2
60   1
61   1
63   1
66   2
68   2
69   1
71   1
73   1
76   2
77   1
80   1
81   1
82   2
83   1
84   3
85   2
89   1
92   1
93   3
94   1
97   2
98   2
100   1
101   3
102   3
103   1
106   1
108   1
109   1
110   2
113   1
115   1
116   1
117   1
119   2
120   1
121   3
122   2
125   1
126   1
128   1
129   1
131   2
132   1
133   3
135   2
141   1
143   1
145   1
146   1
147   2
149   1
150   1
151   1
152   1
154   1
155   1
163   1
164   2
166   1
167   1
169   3
171   1
174   1
175   1
176   1
177   2
178   2
179   2
183   1
185   2
189   1
190   2
196   1
201   1
206   1
207   1
208   2
211   1
214   1
215   1
216   1
217   1
219   1
220   1
221   2
223   1
224   1
226   1
229   1
231   1
233   1
234   1
236   1
252   1
258   1
264   1
272   1
281   1
285   1
287   1
291   1
296   1
298   1
302   2
303   1
306   1
312   1
313   1
322   1
323   1
327   2
330   1
339   2
349   1
352   1
358   1
362   2
375   1
393   1
395   1
406  

In [14]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [15]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config
    )


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [16]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False, gradient_checkpointing_kwargs={'use_reentrant':False})

In [17]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)

In [18]:
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 617,433,088 || trainable%: 0.3821


In [19]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=lora_model,
    pad_to_multiple_of=8
    )

In [20]:
from evaluate import load

metric = load("sacrebleu")

In [21]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    for i in range(len(labels)):
        labels[i] = [tokenizer.pad_token_id if j<0 else j for j in labels[i]]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

### **Training**

In [22]:
from transformers import Seq2SeqTrainingArguments

batch_size = 2

args = Seq2SeqTrainingArguments(
    output_dir="./",
    run_name="finetuning_sampled_NLLB",
    eval_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
)

In [23]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    lora_model,
    args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [24]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,0.678980,46.326700,76.906200
2,No log,0.497843,65.666900,62.565700
3,No log,0.477361,68.179200,61.655500


Se han truncado las últimas 5000 líneas del flujo de salida.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class in

TrainOutput(global_step=300, training_loss=0.7845369466145833, metrics={'train_runtime': 7033.2965, 'train_samples_per_second': 0.085, 'train_steps_per_second': 0.043, 'total_flos': 401533727735808.0, 'train_loss': 0.7845369466145833, 'epoch': 3.0})

In [25]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_pretrained(
    checkpoint,
)

print(generation_config)

GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "eos_token_id": 2,
  "max_length": 200,
  "pad_token_id": 1
}



### **Evaluación**

In [27]:
test_batch_size = 32
batch_tokenized_test = tokenized_test_dataset.batch(test_batch_size)

Batching examples:   0%|          | 0/978 [00:00<?, ? examples/s]

In [28]:
number_of_batches = len(batch_tokenized_test["code"])
output_sequences = []
for i in range(number_of_batches):
    inputs = tokenizer(batch_tokenized_test["code"][i], max_length=max_tok_length,
                       truncation=True, return_tensors="pt", padding=True)
    output_batch = lora_model.generate(generation_config=generation_config,
                                       input_ids=inputs["input_ids"].cuda(),
                                       attention_mask=inputs["attention_mask"].cuda(),
                                       forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_code),
                                       max_length = max_tok_length, num_beams=1, do_sample=False,)
    output_sequences.extend(output_batch.cpu())

In [29]:
decoded_preds = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

In [30]:
references = tokenizer.batch_decode(tokenized_test_dataset["labels"], skip_special_tokens=True)

In [31]:
sources = test_dataset_filtered['code']

In [32]:
from whisper.normalizers.basic import BasicTextNormalizer

normalizer = BasicTextNormalizer()

decoded_preds_clean = [normalizer(text) for text in decoded_preds]
references_clean = [normalizer(text) for text in references]
sources_clean = [normalizer(text) for text in sources]

In [33]:
from evaluate import load

metric = load("sacrebleu")

In [34]:
result = metric.compute(predictions=decoded_preds_clean, references=references_clean)
print(f'BLEU score: {result["score"]:.1f}')

BLEU score: 67.4


In [35]:
from evaluate import load

rouge_metric = load("rouge")
comet_metric = load("comet")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [36]:
comet_score = comet_metric.compute(predictions = decoded_preds_clean, references = references_clean, sources = sources_clean)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [37]:
print(f"COMET: {comet_score['mean_score'] * 100:.2f} %")

COMET: 85.08 %


In [38]:
rouge_score = rouge_metric.compute(predictions = decoded_preds_clean, references = references_clean)

In [39]:
print(f"Rouge 1: {rouge_score['rouge1'] * 100:.2f} %")
print(f"Rouge 2: {rouge_score['rouge2'] * 100:.2f} %")
print(f"Rouge L: {rouge_score['rougeL'] * 100:.2f} %")

Rouge 1: 86.88 %
Rouge 2: 84.75 %
Rouge L: 86.64 %
